In [2]:
import sys 

# Install all dependencies
#!{sys.executable} -m pip install numpy scipy matplotlib pyvis 

# Change path as the folder adress you have put the library in 
path = "C:\\Users\\Paul Valcke\\Documents\\GitHub\\GEMMES" 
sys.path.insert(0, path)
import pygemmes as pgm
import plotly.graph_objects as go
import numpy as np
import pandas as pd


#####################################################################################
 _______            ______  ________  ____    ____  ____    ____  ________   ______    
|_   __ \         .' ___  ||_   __  ||_   \  /   _||_   \  /   _||_   __  |.' ____  \   
  | |__) |_   __ / .'   \_|  | |_ \_|  |   \/   |    |   \/   |    | |_ \_|| (___ \_|  
  |  ___/[ \ [  ]| |   ____  |  _| _   | |\  /| |    | |\  /| |    |  _| _  _.____`.   
 _| |_    \ '/ / \ `.___]  |_| |__/ | _| |_\/_| |_  _| |_\/_| |_  _| |__/ || \____) |  
|_____| [\_:  /   `._____.'|________||_____||_____||_____||_____||________| \______.'  
         \__.'                           
#####################################################################################                                                                                                                                   

Welcome in Pygemmes, a modular library to prototype and study dynamical systems !
This library is oriented toward generation

In [5]:

hub=pgm.Hub('ECHIMES',preset='SimpleTri',verb=False)
hub.run()
pgm.get_available_models()

time step 467 / 500
time step 500 / 500


,Folder,Preset
TUTORIAL,,[presetname1]
Climate_3Layers,Components,[default]
Goodwin,Core,"[default, many-orbits]"
GoodwinFull,Core,"[default, many-orbits]"
Accessibility,ECHIMES,[3sectors]
CHIMES0,ECHIMES,"[Goodwin, Bisectoral, SimpleBi, SimpleTri, Sim..."
ECHIMES,ECHIMES,"[Goodwin, Bisectoral, SimpleBi, SimpleTri, Sim..."
InternationalExchange,ECHIMES,[3by2]
logisticConsumption,ECHIMES,[]
MultisectoralAggregates,ECHIMES,[]


In [ ]:
#hub.plot(filters_key=['pi'])

In [47]:

listX=['Minvest','Minter']
nx=0
region=0
ntindex=0
R=hub.get_dparam()

c = [ 'rgba(255,0,255, 0.8)' , 'rgba(0,255,255, 0.8)' , 'rgba(255,255,0, 0.8)','rgba(127,255,127, 0.8)' ]

### INITIAL LISTS TO FILL
source=[] # Where the flux starts
target=[] # Where the flux ends
value =[] # Flux intensity
types =[]
colors=[]
label =[] 

### MATRICES
for ii,X in enumerate(listX):
    d0=R[X]
    values = d0['value'][ntindex,nx,region,:,:].reshape(-1)
    names = R[d0['size'][0]]['list']
    sectintdex=np.arange(len(names))

    XX,YY=np.meshgrid(sectintdex+1,sectintdex+1)
    XX=XX.astype(int).reshape(-1)-1
    YY=YY.astype(int).reshape(-1)-1
    Ntot=len(XX)
    
    source.extend(YY)
    target.extend(XX)
    types.extend([X for i in range(len(XX))])
    value.extend(values)
    colors.extend([c[ii] for j in range(len(XX))])
    label = names


### ADDING VECTORS
Vecid = 'C'
stype = 'Consumption'
slabel = 'Household'
V=R[Vecid]['value'][ntindex,nx,region,:,0]        # Values we add 
#if slabel not in label:
label.append(slabel)
target.extend([np.amax(target+source)+1]*len(V))
source.extend(np.arange(0,len(V)))
value.extend(V)
types.extend([stype for i in range(len(V))])
colors.extend([c[ii+1] for ii in range(len(V))])

tempdict={          #'label':label,
                   'target':target,
                   'source':source,
                   'value':value,
                   'types':types,
                   'colors':colors}
df = pd.DataFrame(tempdict)


link = dict(source = np.array(source).reshape(-1), 
                             target = np.array(target).reshape(-1), 
                             value = np.array(value).reshape(-1),
                             color=colors)
node = dict(  label = label,
                             pad=50, 
                             thickness=5)
data = go.Sankey(link = link , 
                 node= node)
# plot
fig = go.Figure(data)
fig.update_layout(
    hovermode = 'x',
    title=f"Physical exchanges between sectors, t={R['time']['value'][ntindex,0,0,0,0]:.2f}",
    font=dict(size = 10, color = 'white'),
    paper_bgcolor='#5B5958'
)
fig.show()

In [45]:
#### MONETARY FLUXES 
Matrices=['MtransactI','MtransactY']
Vectors =['wL','pC','rD']
nx=0
region=0
ntindex=-1
R=hub.get_dparam()

c = [ 'rgba(255,  0,255, 0.8)' , 
      'rgba(  0,255,255, 0.8)' ,
      'rgba(255,255,  0, 0.8)' ,
      'rgba(127,255,127, 0.8)' ,
      'rgba(0,0,0, 0.8)' ,
      'rgba(127,255,255, 0.8)' ,
      'rgba(255,127,255, 0.8)']

### INITIAL LISTS TO FILL
TD={               'label' :[], # names of target/sources
                   'target':[], # Where the flux ends
                   'source':[], # Where the flux starts
                   'value' :[], # Flux intensity
                   'types' :[], # Flux category 
                   'colors':[]}

def Add_Matrix(X):
    d0=R[X]
    values = d0['value'][ntindex,nx,region,:,:].reshape(-1)
    names = R[d0['size'][0]]['list']
    sectintdex=np.arange(len(names))

    XX,YY=np.meshgrid(sectintdex+1,sectintdex+1)
    XX=XX.astype(int).reshape(-1)-1
    YY=YY.astype(int).reshape(-1)-1
    Ntot=len(XX)
    
    TD['source'].extend(YY)
    TD['target'].extend(XX)
    TD['types'].extend([X for i in range(len(XX))])
    TD['value'].extend(values)
    TD['colors'].extend([len(set(TD['types']))-1 for j in range(len(XX))])
    TD['label'] = names
    return TD

def Add_Vector(TD,Vecid,stype,slabel,switch):
    V=R[Vecid]['value'][ntindex,nx,region,:,0]        # Values we add 
    if slabel not in TD['label']:
        TD['label'].append(slabel)
        plus= 1
    else :
        plus=0
    V1 = np.arange(0,len(V))
    V2 = [np.amax(TD['target']+TD['source'])+plus]*len(V)
    TD['target'].extend(V1 if switch else V2) 
    TD['source'].extend(V2 if switch else V1)
    TD['value'].extend(V)
    TD['types'].extend([stype for i in range(len(V))])
    TD['colors'].extend([len(set(TD['colors'])) for ii in range(len(V))])
    return TD

def Add_scalar(TD,name,source,target,type,newcolor=False):
    print(source,TD['label'])
    if source in TD['label']:
        sourceindex = TD['label'].index(source)
    else : 
        print(source,'adding')
        sourceindex = len(TD['label'])
        TD['label'].append(source)

    if target in TD['label']:
        targetindex = TD['label'].index(target)
    else : 
        targetindex = len(TD['label'])
        TD['label'].append(target)

    TD['source'].append(sourceindex)
    TD['target'].append(targetindex)
    TD['value'].append(R[name]['value'][ntindex,nx,region,0,0])
    TD['types'].append(type)
    TD['colors'].append(TD['colors'][-1]+1 if newcolor else TD['colors'][-1])
    return TD    


### MATRICES
for ii,X in enumerate(Matrices):
    Add_Matrix(X)



### ADDING VECTORS
Vecid = 'pC'
stype = 'Consumption'
slabel = 'Household'
switch = True
TD = Add_Vector(TD,Vecid,stype,slabel,switch)

Vecid = 'wL'
stype = 'Wages'
slabel = 'Household'
TD = Add_Vector(TD,Vecid,stype,slabel,False)

Vecid = 'rD'
stype = 'Interests'
slabel = 'Banks'
TD = Add_Vector(TD,Vecid,stype,slabel,False)

TD = Add_scalar(TD,'rDh','Household','Banks','Interests',False)


TD['colors']= [c[i] for i in TD['colors']]
for i,v in enumerate(TD['value']):
    if v<0:
        TD['target'][i],TD['source'][i] = TD['source'][i],TD['target'][i]
        TD['value'][i]*=-1


df = pd.DataFrame({k:v for k,v in TD.items() if k!='label'})


link = dict(source = np.array(TD['source']).reshape(-1), 
                             target = np.array(TD['target']).reshape(-1), 
                             value = np.array(TD['value']).reshape(-1),
                             color=TD['colors'])
node = dict(  label = TD['label'],
                             pad=50, 
                             thickness=5)
data = go.Sankey(link = link , 
                 node= node)
# plot
figMoney = go.Figure(data)
figMoney.update_layout(
    hovermode = 'x',
    title=f"Monetary exchanges between sectors, t={R['time']['value'][ntindex,0,0,0,0]:.2f}",
    font=dict(size = 10, color = 'white'),
    paper_bgcolor='#5B5958'
)
figMoney.show()

Household ['Consumption', 'Intermediate', 'Capital', 'Household', 'Banks']


In [46]:
df

,target,source,value,types,colors
0,0,0,0.00e+00,MtransactI,"rgba(255, 0,255, 0.8)"
1,1,0,0.00e+00,MtransactI,"rgba(255, 0,255, 0.8)"
2,2,0,2.05e+00,MtransactI,"rgba(255, 0,255, 0.8)"
3,0,1,0.00e+00,MtransactI,"rgba(255, 0,255, 0.8)"
4,1,1,0.00e+00,MtransactI,"rgba(255, 0,255, 0.8)"
5,2,1,4.01e-01,MtransactI,"rgba(255, 0,255, 0.8)"
6,0,2,0.00e+00,MtransactI,"rgba(255, 0,255, 0.8)"
7,1,2,0.00e+00,MtransactI,"rgba(255, 0,255, 0.8)"
8,2,2,1.63e+00,MtransactI,"rgba(255, 0,255, 0.8)"
9,0,0,0.00e+00,MtransactY,"rgba( 0,255,255, 0.8)"


In [1]:
import ipywidgets

In [2]:
slider = ipwidgets.IntSlider(
    min=0,
    max=10,
    step=1,
    description='Sankey time index :',
    value=3
)

SyntaxError: incomplete input (1712168102.py, line 6)